In [1]:
import cirq
from math import pi

In [2]:
# Assigning qubits that will be required by the circuit
MAX_QUBITS = 5 # We need to know this number before hand to compile an equivalent circuit
N_QUBITS = MAX_QUBITS
qubits = [cirq.LineQubit(i) for i in range(N_QUBITS)]

In [8]:
# Creating a sample circuit
circuit = cirq.Circuit()

# Applying Hadamard Gate
circuit.append([cirq.H(qubits[0])])
circuit.append([cirq.H(qubits[1])])
circuit.append([cirq.H(qubits[2])])

# Applying CNOT Gate
for i in range(N_QUBITS - 1):
    circuit.append([cirq.CNOT(qubits[i], qubits[i + 1])], cirq.InsertStrategy.NEW)

# Shifting phase along x axis by pi/2 radians
phase_shift = cirq.rx(pi / 2)
for q in qubits:
    circuit.append([phase_shift(q)], cirq.InsertStrategy.INLINE)

# Adding a measurement gate
# for q in qubits:
#     circuit.append([cirq.measure(q)])

In [4]:
# Printing the circuit
print(circuit)

0: ───H───@───────────Rx(0.5π)──────────────
          │
1: ───H───X───@───────Rx(0.5π)──────────────
              │
2: ───H───────X───@───Rx(0.5π)──────────────
                  │
3: ───────────────X───@──────────Rx(0.5π)───
                      │
4: ───────────────────X──────────Rx(0.5π)───


In [15]:
# Available gates for the new compiled circuit
def compile_rx(qubit, theta):
    rx = cirq.rx(theta)
    return rx(qubit)


def compile_rz(qubit, theta):
    rz = cirq.rz(theta)
    return rz(qubit)


def compile_CZ(control_q, target_q):
    return cirq.CZ(control_q, target_q)


# Gates that were used by the input circuit apart from the ones above
def compile_ry(qubit, theta):
    return compile_rx(qubit, pi / 2), compile_rz(qubit, theta), compile_rx(qubit, -pi / 2)


def compile_I(qubit):
    return compile_rx(qubit, 2 * pi)


def compile_H(qubit):
    return compile_rz(qubit, pi / 2), compile_rx(qubit, pi / 2), compile_rz(qubit, pi / 2)


def compile_X(qubit):
    return compile_rx(qubit, pi)


def compile_Y(qubit):
    return compile_ry(qubit, pi)


def compile_Z(qubit):
    return compile_rz(qubit, pi)


def compile_CNOT(control_q, target_q):
    return compile_H(target_q), compile_CZ(control_q, target_q), compile_H(target_q)

In [16]:
def get_compiled_ops(moment):
    compiled_operations = []
    
    # For each operation in a Moment
    for op in moment.operations:

        # Retrieving which gate operation is being performed
        gate = op.gate

        # Compiling gates accordingly:

        # Single qubit gates
        if gate == cirq.I:
            new_op = compile_I(op.qubits[0])

        elif gate == cirq.H:
            new_op = compile_H(op.qubits[0])

        elif gate == cirq.X:
            new_op = compile_X(op.qubits[0])

        elif gate == cirq.Y:
            new_op = compile_Y(op.qubits[0])

        elif gate == cirq.Z:
            new_op = compile_Z(op.qubits[0])
        
        # Two qubit gates
        elif gate == cirq.CNOT:
            new_op = compile_CNOT(op.qubits[0], op.qubits[1])

        elif gate == cirq.CZ:
            new_op = compile_CZ(op.qubits[0], op.qubits[1])
        
        # Single qubit rotation gates along x, y, z axis on Bloch sphere
        # with rotation by theta radians
        else:

            # Retrieveing the value of theta
            theta = gate.exponent * pi

            if gate == cirq.rx(theta):
                new_op = compile_rx(op.qubits[0], theta)

            elif gate == cirq.ry(theta):
                new_op = compile_ry(op.qubits[0], theta)

            elif gate == cirq.rz(theta):
                new_op = compile_rz(op.qubits[0], theta)

        
        compiled_operations.append(new_op)
        
    return compiled_operations

In [21]:
new_circuit = cirq.Circuit()
# rx_shift = cirq.rx(pi)
# circuit = cirq.Circuit([rx_shift(qubits[0])])
for moment in circuit:
    compiled_operations = get_compiled_ops(moment)
    new_circuit.append(compiled_operations)
print(new_circuit)

0: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)────────────────────────────────────@───Rx(0.5π)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                        │
1: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)──────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                             │
2: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)────────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)─────────────────────────────────────────────────────────────────────────
                                                                                                                       

As it can be seen, the number of moments in the input circuit was 6 but in the output circuit it is 23. This increase in the number of moments can be considered as overhead generated by our compiler. So let's see how we can decrease this number.

In [22]:
print("Number of Moments in the input circuit: ", len(circuit))
print("Number of Moments in the compiled output circuit: ", len(new_circuit))

Number of Moments in the input circuit:  6
Number of Moments in the compiled output circuit:  23


In [24]:
# Gate combinations that can be merged to I
can_be_merged_to_i = [
    [cirq.rx(2 * pi)],                                                                 # equivalent to --X-X--
    [cirq.rx(pi / 2), cirq.rz(pi), cirq.rx(0), cirq.rz(pi), cirq.rx(-pi / 2)],         # equivalent to --Y-Y--
    [cirq.rz(2 * pi)],                                                                 # equivalent to --Z-Z--
    [cirq.rx(3 * pi / 2), cirq.rz(pi), cirq.rx(-pi / 2), cirq.rz(pi / 2)],             # equivalent to --X-Y-Z--
    [cirq.rz(pi / 2), cirq.rx(pi / 2), cirq.rz(pi), cirq.rx(pi / 2), cirq.rz(pi / 2)]  # equivalent to --H-H--
]

In [59]:
def optimize(circuit):
    moments_of_q = []
    
    for q in qubits:
        moq = [moment[[q]] for moment in circuit]               # Moments of q -> Moments where an operation was performed on qubit "q"
        cirq.DropEmptyMoments().optimize_circuit(moq)
        moments_of_q.append(moq)
    
    
    for moq in moments_of_q:
        ops = [moment.operations[0].gate for moment in moq]
        print(ops)

    return
optimize(new_circuit)

[cirq.rz(np.pi*0.5), cirq.rx(np.pi*0.5), cirq.rz(np.pi*0.5), cirq.CZ, cirq.rx(np.pi*0.5)]
[cirq.rz(np.pi*0.5), cirq.rx(np.pi*0.5), cirq.rz(np.pi*0.5), cirq.rz(np.pi*0.5), cirq.rx(np.pi*0.5), cirq.rz(np.pi*0.5), cirq.CZ, cirq.rz(np.pi*0.5), cirq.rx(np.pi*0.5), cirq.rz(np.pi*0.5), cirq.CZ, cirq.rx(np.pi*0.5)]
[cirq.rz(np.pi*0.5), cirq.rx(np.pi*0.5), cirq.rz(np.pi*0.5), cirq.rz(np.pi*0.5), cirq.rx(np.pi*0.5), cirq.rz(np.pi*0.5), cirq.CZ, cirq.rz(np.pi*0.5), cirq.rx(np.pi*0.5), cirq.rz(np.pi*0.5), cirq.CZ, cirq.rx(np.pi*0.5)]
[cirq.rz(np.pi*0.5), cirq.rx(np.pi*0.5), cirq.rz(np.pi*0.5), cirq.CZ, cirq.rz(np.pi*0.5), cirq.rx(np.pi*0.5), cirq.rz(np.pi*0.5), cirq.CZ, cirq.rx(np.pi*0.5)]
[cirq.rz(np.pi*0.5), cirq.rx(np.pi*0.5), cirq.rz(np.pi*0.5), cirq.CZ, cirq.rz(np.pi*0.5), cirq.rx(np.pi*0.5), cirq.rz(np.pi*0.5), cirq.rx(np.pi*0.5)]


AttributeError: 'NoneType' object has no attribute 'gate'